In [13]:
import torch
from torch.autograd import Variable

In [14]:
def to_variable(x):
  if torch.cuda.is_available():
    x = x.cuda()
  return Variable(x)

In [15]:
import torchvision.transforms as transforms
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize(mean=0.5, std=0.5)
])

In [16]:
from torchvision.datasets import MNIST
train_dataset = MNIST(root = './', train=True, download=True, transform=transform)
test_dataset= MNIST(root = './', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [18]:
import torch.utils.data as Data
data_loader = Data.DataLoader(dataset=train_dataset, 
                               batch_size=100, 
                               shuffle=True)

In [19]:
import torch.nn as nn

In [20]:
D = nn.Sequential(
    nn.Linear(28*28, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid()
)

In [21]:
G = nn.Sequential(
    nn.Linear(64, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 28*28),
    nn.Tanh() # 1 ~ -1
)

In [22]:
if torch.cuda.is_available():
    D.cuda()
    G.cuda()

In [23]:
loss_func = nn.BCELoss()
d_opt = torch.optim.Adam(D.parameters(), lr=0.0003)
g_opt = torch.optim.Adam(G.parameters(), lr=0.0003)

In [24]:
from torchvision.utils import save_image

In [25]:
def denorm(x):
  # 平均0.5 三個標準差 為 -1, 2
  out = (x+1)/2
  return out.clamp(0, 1)

In [27]:
for epoch in range(200):
  for i, (images, _) in enumerate(data_loader):
    batch_size = images.size(0)
    images = to_variable(images.view(batch_size, -1))
    
    real_labels = to_variable(torch.ones(batch_size, 1))
    fake_labels = to_variable(torch.zeros(batch_size, 1))

    outputs = D(images)
    d_loss_real = loss_func(outputs, real_labels)
    real_score = outputs

    z = to_variable(torch.randn(batch_size, 64))
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = loss_func(outputs, fake_labels)
    fake_score = outputs

    d_loss = d_loss_real + d_loss_fake
    D.zero_grad()
    d_loss.backward()
    d_opt.step()

    z = to_variable(torch.randn(batch_size, 64))
    fake_images = G(z) 
    outputs = D(fake_images)

    g_loss = loss_func(outputs, real_labels)
    D.zero_grad()
    G.zero_grad()
    g_loss.backward()
    g_opt.step()

    if (i+1)%300 == 0:
      print('Epoch %d, batch %d, d_loss %.4f g_loss %.4f'
      ' D(x): %.2f, G(z): %.2f '
      %(epoch, i+1, d_loss.data, g_loss.data, real_score.mean().data.mean(), fake_score.data.mean()))

    if epoch == 0:
      images = images.view(images.size(0), 1, 28, 28)
      save_image(denorm(images), './data/real_images.png')
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28) # c, w, h
    save_image(denorm(fake_images), './data/fake_images-%d.png'%(epoch+1))



Epoch 0, batch 300, d_loss 0.2833 g_loss 5.3342 D(x): 0.98, G(z): 0.20 
Epoch 0, batch 600, d_loss 0.3007 g_loss 4.4066 D(x): 0.93, G(z): 0.17 
Epoch 1, batch 300, d_loss 0.3905 g_loss 3.4617 D(x): 0.89, G(z): 0.20 
Epoch 1, batch 600, d_loss 0.8105 g_loss 1.4808 D(x): 0.76, G(z): 0.28 
Epoch 2, batch 300, d_loss 1.9072 g_loss 0.8031 D(x): 0.48, G(z): 0.51 
Epoch 2, batch 600, d_loss 0.4866 g_loss 2.7882 D(x): 0.88, G(z): 0.23 
Epoch 3, batch 300, d_loss 1.7211 g_loss 2.1081 D(x): 0.70, G(z): 0.49 
Epoch 3, batch 600, d_loss 0.5888 g_loss 3.7402 D(x): 0.80, G(z): 0.13 
Epoch 4, batch 300, d_loss 1.0428 g_loss 2.0345 D(x): 0.72, G(z): 0.37 
Epoch 4, batch 600, d_loss 0.7371 g_loss 3.4148 D(x): 0.76, G(z): 0.20 
Epoch 5, batch 300, d_loss 1.0581 g_loss 1.1345 D(x): 0.72, G(z): 0.46 
Epoch 5, batch 600, d_loss 1.2362 g_loss 1.2082 D(x): 0.65, G(z): 0.46 
Epoch 6, batch 300, d_loss 1.1382 g_loss 1.6572 D(x): 0.69, G(z): 0.39 
Epoch 6, batch 600, d_loss 0.9883 g_loss 1.5772 D(x): 0.80, G(z)